In [1]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import math

# load the train data
dataframe = read_csv('dataset/train.csv',usecols=[i for i in range(0,211)], engine='python', header=0)
trainData = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

In [2]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [3]:
#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]
# normalize the dataset
scalerInput = MinMaxScaler(feature_range=(0, 1))
scalerOutput = MinMaxScaler(feature_range=(0, 1))

trainX = scalerInput.fit_transform(trainX)
trainY = scalerOutput.fit_transform(trainY)
testX = scalerInput.fit_transform(testX)

#Put it into batches
trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1]))

print trainX.shape
print trainY.shape
print testX.shape

(625, 64, 146)
(625, 64, 64)
(1875, 64, 146)


In [4]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from keras.models import Model

class Model:
    
    def __init__(self, trainX, trainY):
        self.optimizer = adam(0.01, 0.01, 0.001, 1e-08, 0.0)
        self.model = Sequential()
        self.model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
        self.model.add(Dense(output_dim, activation='linear'))
        #Configures the model for training.
        self.model.compile(loss="mean_squared_error",optimizer=self.optimizer)
        #Trains the model for a fixed number of epochs (iterations on a dataset).
        self.model.fit(trainX,trainY,batch_size=batch_size,epochs=1)
        #Make predictions
    
    def predict(self,testX):
        return self.model.predict(testX)
    

Using TensorFlow backend.


In [5]:
#train the model
nn = Model(trainX, trainY)
testY = nn.predict(testX)

Epoch 1/1
625/625 [==============================] - 3s - loss: 0.0968     


In [19]:
testY = testY.reshape(len(testData),output_dim)
testY = scalerOutput.inverse_transform(testY)
print testY


[[ -2.25258470e-02  -2.28753686e-02  -2.16194876e-02 ...,  -6.20678008e-01
    2.19840460e+12   2.53691350e+12]
 [ -2.21500136e-02  -2.27742940e-02  -2.14728676e-02 ...,  -5.83657146e-01
    2.20374709e+12   2.57295699e+12]
 [ -2.20421813e-02  -2.27500163e-02  -2.15244908e-02 ...,  -5.74804306e-01
    2.24092409e+12   2.62375395e+12]
 ..., 
 [ -2.20589526e-02  -2.27065384e-02  -2.16787029e-02 ...,  -5.80184162e-01
    2.25153909e+12   2.54162292e+12]
 [ -2.20447853e-02  -2.27398518e-02  -2.15612538e-02 ...,  -5.78059971e-01
    2.22378197e+12   2.57035521e+12]
 [ -2.20487919e-02  -2.27191467e-02  -2.16432773e-02 ...,  -5.70059180e-01
    2.21202429e+12   2.55801190e+12]]


In [20]:
#create the result.txt
f1=open('./result.txt', 'w+')
f1.write("Id,Predicted\n")
for y in range(5):
    for x in range(testY.shape[1]-2):
        f1.write(str(y+1)+"_"+str(x+1)+","+str(testY[y][x])+"\n")
f1.close()